<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW1_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日常支出速算與分攤（作業一）
*   目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA 分攤 → 寫回 Sheet Summary 分頁。
*   AI 點子（可選）：請模型總結本週花錢習慣與建議（例如「外食過多」）。
*   Sheet 欄位：date, category, item, amount, payer



In [16]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing')


# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

import datetime
# 讓使用者輸入資料
date_str = input("請輸入日期 (格式：YYYY-MM-DD): ")
# 檢查日期格式是否正確
datetime.datetime.strptime(date_str, '%Y-%m-%d')

time_str = input("請輸入時間 (格式：HH:MM): ")
# 檢查時間格式是否正確
datetime.datetime.strptime(time_str, '%H:%M')

item = input("請輸入品項: ")
amount = float(input("請輸入金額: "))

# 創建一個新的 DataFrame 來代表你要新增的資料
new_data = pd.DataFrame([{
    '日期': date_str,
    '時間': time_str,
    '品項': item,
    '金額': amount
}])

# 使用 concat() 將新資料合併到舊的 df 中
df = pd.concat([df, new_data], ignore_index=True)

# 將 DataFrame 轉換成列表的列表 (list of lists)，這是 gspread 支援的資料格式
data_to_write = df.values.tolist()

# 第一步：取得工作表物件
worksheet = gsheets.worksheet('工作表1')

# 第二步：將資料寫入工作表
worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
print("資料已成功寫入 Google 工作表！")

請輸入日期 (格式：YYYY-MM-DD): 2025-09-11
請輸入時間 (格式：HH:MM): 10:30
請輸入品項: 早餐
請輸入金額: 100


/tmp/ipython-input-309751333.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_data], ignore_index=True)


資料已成功寫入 Google 工作表！
